In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
Rx = PolynomialRing(QQ, 'x')
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)

def divides(a,b):
    if mod(b,a)==0:return True
    return False

def ord_p_n_nonnegative(p,n):
    if (n in ZZ)==False:return(n,"is not an integer")
    if n==0:return Infinity
    if is_prime(p)==False:return(p," is not prime.")
    if gcd(p,n)==1:
        return 0
    
    for k in [0..n]:
        if divides(p^k,n)==True:
            if divides(p^(k+1),n)==False:
                return k
        
def ord_p(p,n):return ord_p_n_nonnegative(p,abs(n))


def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H(n,m): # definition 8.1 (13aug21)
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    power = 1/(m-2)
    frac=numerator/denominator
    return ((S(frac^power)).O(n+1)).polynomial()

def H4(n,m): #per dfmn 8.2 in 2021 Integer paper
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def deltaDiamond(n,m): #from 'conjecture 9.ipynb' (interpolation paper)
    return D(S(expand((H(n+2,m)^3)/J(n+2,m)))).truncate(n+1)

def deltaDiamondWt12(n,m):
    return D(S(expand((H4(n+2,m)^3)/J(n+2,m)))).truncate(n+1)

def deltaDiamondWt12Strike(n,m): #this is the cusp form from cn 9 of the interpolations paper
    # substitute 2^6*m^3*x for x in def 9.1; not
    # yet defined in draft  21h28jul21
    f=deltaDiamondWt12(n,m)
    fl=f.list()
    df=f.degree()
    sm=sum(fl[k]*(x*m^3*2^6)^k for k in [0..df])
    dsm=sm.degree()
    sml=sm.list()
    notzero=[]
    for entry in sml:
        if (entry!=0):notzero=notzero+[entry]
    leadingterm=notzero[0]
    sm2=sum(fl[k]*(x*m^3*2^6)^k/leadingterm for k in [0..dsm])
    return sm2

def reciprocalDeltaDiamondWt12(n,m):
    num=J(n+2,m)
    den=H4(n+2,m)^3
    ex=expand(num/den)
    return ex.truncate(n+1)

def reciprocalDeltaDiamondWt12Strike(n,m):
    rd=reciprocalDeltaDiamondWt12(n,m)
    trl=rd.list()
    lt=len(trl)
    trl_strike=2^6*m^3*vector(trl)
    def f(n):return x^n # 
    variables=vector([f(n) for n in range(-1,lt-1)])
    answer=variables.dot_product(trl_strike)
    return answer

def reciprocalDeltaDiamondWt12_Strike(n,m):
    # this is *not* the reciprocal of deltaDiamondWt12_StrikeC !!
    # rather it is the image under the strike operation of
    # reciprocalDeltaDiamondWt12 ....
    rd=reciprocalDeltaDiamondWt12(n,m)
    trl=rd.list()
    lt=len(trl)
    trl_strike=2^6*m^3*vector(trl)
    def f(n):return L(x^n) # L coercion is necessary
    variables=vector([f(n) for n in range(-1,lt-1)])
    answer=variables.dot_product(trl_strike)
    return answer
        
def reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power):
    n=power
    trun=reciprocalDeltaDiamondWt12_Strike(n,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

P=Primes()
def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def multiplicative_order_integer(p,n):
    fi=factor_integer(n)
    for pair in fi:
        if pair[0]==p:return pair[1]
    return 0

def multiplicative_order_integerOld(p,n):
    tv=[]
    answers=[0]
    for k in [0..20]:
        old=tv
        tv=tv+[n/p^k in ZZ]
        if (False in tv)&((False in old)==False):answers=answers+[k-1]
    return max(answers)

def multiplicative_order(d,n): 
    n=QQ(n)
    num=ZZ(numerator(n))
    den=ZZ(denominator(n))
    on=multiplicative_order_integer(d,num)
    od=multiplicative_order_integer(d,den)
    return on-od

def multiplicative_orderOld(d,n): #apparently, Old version is faster sometimes.
    n=QQ(n)
    num=ZZ(numerator(n))
    den=ZZ(denominator(n))
    on=multiplicative_order_integerOld(d,num)
    od=multiplicative_order_integerOld(d,den)
    return on-od



def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jNormalizedPowerConstantTerm(m,power,change):
    trun=L(j(power,m)-QQ(change))
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [83]:
for m in [3..6]:
    print("-----------------------------------------------------------------------")
    print((reciprocalDeltaDiamondWt12(5,m)*deltaDiamondWt12(5,m)).O(5))

-----------------------------------------------------------------------
1 + O(x^5)
-----------------------------------------------------------------------
1 + O(x^5)
-----------------------------------------------------------------------
1 + O(x^5)
-----------------------------------------------------------------------
1 + O(x^5)


In [84]:
def reciprocalDeltaDiamondWt12_Strike(n,m):
    # this is *not* the reciprocal of deltaDiamondWt12_StrikeC !!
    # rather it is the image under the strike operation of
    # reciprocalDeltaDiamondWt12 ....
    rd=reciprocalDeltaDiamondWt12(n,m)
    trl=rd.list()
    lt=len(trl)
    trl_strike=2^6*m^3*vector(trl)
    def f(n):return L(x^n) # L coercion is necessary
    variables=vector([f(n) for n in range(-1,lt-1)])
    answer=variables.dot_product(trl_strike)
    return answer

In [85]:
def reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power):
    n=power
    trun=reciprocalDeltaDiamondWt12_Strike(n,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

print(reciprocalDeltaDiamondWt12Strike_ConstantTerm3(5,3))

11474728000


In [87]:
power=1
for m in [3..10]:
    print("-----------------------------------------------------------------------")
    print(reciprocalDeltaDiamondWt12_Strike(power,m)^power);print()
    cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
    print("constant: ",cst)

-----------------------------------------------------------------------
1728*x^-1 + 24 + 3/16*x

constant:  24
-----------------------------------------------------------------------
4096*x^-1 + 512 + 67/4*x

constant:  512
-----------------------------------------------------------------------
8000*x^-1 + 1480 + 8353/80*x

constant:  1480
-----------------------------------------------------------------------
13824*x^-1 + 3072 + 288*x

constant:  3072
-----------------------------------------------------------------------
21952*x^-1 + 5432 + 9487/16*x

constant:  5432
-----------------------------------------------------------------------
32768*x^-1 + 8704 + 8359/8*x

constant:  8704
-----------------------------------------------------------------------
46656*x^-1 + 13032 + 26713/16*x

constant:  13032
-----------------------------------------------------------------------
64000*x^-1 + 18560 + 12464/5*x

constant:  18560


In [90]:
power=2
for m in [3..10]:
    print("-----------------------------------------------------------------------")
    print(reciprocalDeltaDiamondWt12_Strike(power,m)^power);print()
    cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
    print("constant: ",cst)

-----------------------------------------------------------------------
2985984*x^-2 + 82944*x^-1 + 1224 + 343/27*x + 5387/62208*x^2 + 25/62208*x^3 + 625/544195584*x^4

constant:  1224
-----------------------------------------------------------------------
16777216*x^-2 + 4194304*x^-1 + 399360 + 20736*x + 11657/16*x^2 + 469/32*x^3 + 49/256*x^4

constant:  399360
-----------------------------------------------------------------------
64000000*x^-2 + 23680000*x^-1 + 3861000 + 377017*x + 150232513/6400*x^2 + 141909117/160000*x^3 + 288626121/16000000*x^4

constant:  3861000
-----------------------------------------------------------------------
191102976*x^-2 + 84934656*x^-1 + 17399808 + 59834368/27*x + 44272640/243*x^2 + 753664/81*x^3 + 138674176/531441*x^4

constant:  17399808
-----------------------------------------------------------------------
481890304*x^-2 + 238486528*x^-1 + 55538952 + 8184581*x + 9820141713/12544*x^2 + 4133742049/87808*x^3 + 189858018529/120472576*x^4

constant:  

In [92]:
power=1
test_constants=[]
for m in [3..50]:
    cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
    test_constants=test_constants+[cst/8-(3*m^3-6*m^2-8*m)]
print(test_constants)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [94]:
for m in [3..6]:
    print("-----------------------------------------------------------------------")
    print((reciprocalDeltaDiamondWt12_Strike(1,m)*deltaDiamondWt12Strike(1,m)).O(5))
# Thus they are not reciprocals....

-----------------------------------------------------------------------
1728 + 24*x + 3/16*x^2 + O(x^5)
-----------------------------------------------------------------------
4096 + 512*x + 67/4*x^2 + O(x^5)
-----------------------------------------------------------------------
8000 + 1480*x + 8353/80*x^2 + O(x^5)
-----------------------------------------------------------------------
13824 + 3072*x + 288*x^2 + O(x^5)


In [4]:
polydata=[]
jpolydata=[]
numericalfactors=[]
jnumericalfactors=[]
import time
for power in [1..5]:
    data=[]
    jdata=[]
    for m in [3..3*power+3]:
        start=time.time()
        jcst=jpowerConstantTerm(m,power)
        cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
        data=data+[(m,cst)]
        jdata=jdata+[(m,jcst)]
        finish=time.time()
        print((power,m,cst,jcst,finish-start))
    print()
    rl=R.lagrange_polynomial(data)
    jrl=R.lagrange_polynomial(jdata)
    nf=numericalfactor(rl)
    jnf=numericalfactor(jrl)
    rl=rl.factor()
    jrl=jrl.factor()
    numericalfactors=numericalfactors+[nf/24]
    jnumericalfactors=jnumericalfactors+[jnf/24]
    pair=[power,rl]
    jpair=[power,jrl]
    polydata=polydata+[[pair]]
    jpolydata=jpolydata+[[jpair]]
    print()
    print("(power,nf,jnf): ",(power,nf,jnf));print()
    print(rl);print()
    print(jrl);print()
    print((rl-jrl).factor())
    print("-----------------------------------------------------------------------------------")
print("========================================================================================")
print()
print(numericalfactors) #A005148, just as for constant terms of powers of j(n,m).
print();print(jnumericalfactors)

(1, 3, 24, 744, 0.2210090160369873)
(1, 4, 512, 1664, 0.12145209312438965)
(1, 5, 1480, 3160, 0.2583041191101074)
(1, 6, 3072, 5376, 0.19381403923034668)


(power,nf,jnf):  (1, 24, 24)

(24) * x * (x^2 - 2*x - 8/3)

(24) * x * (x^2 + 4/3)

(-48) * x * (x + 2)
-----------------------------------------------------------------------------------
(2, 3, 1224, 947304, 0.23323273658752441)
(2, 4, 399360, 5007360, 0.12092089653015137)
(2, 5, 3861000, 18561000, 0.2600977420806885)
(2, 6, 17399808, 54558720, 0.2102489471435547)
(2, 7, 55538952, 136273704, 0.31810498237609863)
(2, 8, 144236544, 301768704, 0.138139009475708)
(2, 9, 325623240, 609175080, 0.3190581798553467)


(power,nf,jnf):  (2, 1128, 1128)

(1128) * x^2 * (x^4 - 4*x^3 - 368/141*x^2 + 624/47*x + 1520/141)

(1128) * x^2 * (x^4 + 184/141*x^2 + 80/141)

(-4512) * (x + 2) * x^2 * (x^2 - 48/47*x - 60/47)
-----------------------------------------------------------------------------------
(3, 3, 70080, 1355202240, 0.2390890121459961)
(3,

In [ ]:
jpolydata=[]
numericalfactors=[]
jnumericalfactors=[]
import time
for power in [1..5]:
    data=[]
    jdata=[]
    for m in [3..3*power+3]:
        start=time.time()
        jcst=jpowerConstantTerm(m,power)
        cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
        data=data+[(m,cst)]
        jdata=jdata+[(m,jcst)]
        finish=time.time()
        print((power,m,cst,jcst,finish-start))
    print()
    rl=R.lagrange_polynomial(data)
    jrl=R.lagrange_polynomial(jdata)
    nf=numericalfactor(rl)
    jnf=numericalfactor(jrl)
    rl=rl.factor()
    jrl=jrl.factor()
    numericalfactors=numericalfactors+[nf/24]
    jnumericalfactors=jnumericalfactors+[jnf/24]
    pair=[power,rl]
    jpair=[power,jrl]
    polydata=polydata+[[pair]]
    jpolydata=jpolydata+[[jpair]]
    print()
    print("(power,nf,jnf): ",(power,nf,jnf));print()
    print(rl);print()
    print(jrl);print()
    print((rl-jrl).factor())
    print("-----------------------------------------------------------------------------------")
print("========================================================================================")
print()
print(numericalfactors) #A005148, just as for constant terms of powers of j(n,m).
print();print(jnumericalfactors)

In [7]:
polydata=[]
jpolydata=[]
numericalfactors=[]
jnumericalfactors=[]
no=[]
no_broadhurst=[]
no_jbroadhurst=[]
import time
begin=time.time()
for power in [1..5]:
    data=[]
    jdata=[]
    for m in [3..3*power+3]:
        start=time.time()
        jcst=jpowerConstantTerm(m,power)
        cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
        data=data+[(m,cst)]
        jdata=jdata+[(m,jcst)]
        finish=time.time()
        print((power,m,(finish-start)/60,(finish-begin)/60))
    print()
    rl=R.lagrange_polynomial(data)
    jrl=R.lagrange_polynomial(jdata)
    nf=numericalfactor(rl)
    jnf=numericalfactor(jrl)
    rl=rl.factor()
    jrl=jrl.factor()
    numericalfactors=numericalfactors+[nf/24]
    jnumericalfactors=jnumericalfactors+[jnf/24]
    if nf!=jnf:no=no+[power]
    if nf/24!=broadhurst(power):no_broadhurst=no_broadhurst+[power]
    if jnf/24!=broadhurst(power):no_jbroadhurst=no_jbroadhurst+[power]
    pair=[power,rl]
    jpair=[power,jrl]
    polydata=polydata+[[pair]]
    jpolydata=jpolydata+[[jpair]]
    print()
    print("(power,nf-jnf): ",(power,nf-jnf));print()
    print("(power,nf/24-broadhurst(power): ",(power,nf/24-broadhurst(power)));print()
    print("(power,jnf/24-broadhurst(power): ",(power,jnf/24-broadhurst(power)));print()
    print(rl);print()
    print(jrl);print()
    print((rl-jrl).factor())
    print("-----------------------------------------------------------------------------------")
print("========================================================================================")
print("jnf!=nf:");print(no);print()
print("no_broadhurst:");print(no_broadhurst);print()
print("no_jbroadhurst:");print(no_jbroadhurst);print()
print("numericalfactors:");print()
print(numericalfactors);print()
print("jnumericalfactors:");print()
print(jnumericalfactors)
import pickle
wfile = open('/Users/barrybrent/30oct22no1.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
wjfile = open('/Users/barrybrent/30oct22no2.txt','wb')
tj = pickle.dumps(str(jpolydata))
wjfile.write(tj)
wjfile.close()

(1, 3, 0.003648833433787028, 0.003655298550923665)
(1, 4, 0.0020788153012593585, 0.00574421485265096)
(1, 5, 0.004218284289042155, 0.009964966773986816)
(1, 6, 0.0031787713368733725, 0.013145418961842854)


(power,nf-jnf):  (1, 0)

(power,nf/24-broadhurst(power):  (1, 0)

(power,jnf/24-broadhurst(power):  (1, 0)

(24) * x * (x^2 - 2*x - 8/3)

(24) * x * (x^2 + 4/3)

(-48) * x * (x + 2)
-----------------------------------------------------------------------------------
(2, 3, 0.003889882564544678, 0.017065882682800293)
(2, 4, 0.0019516666730244954, 0.019019114971160888)
(2, 5, 0.004179235299428304, 0.023199915885925293)
(2, 6, 0.0034334341684977214, 0.02663593292236328)
(2, 7, 0.0046123107274373375, 0.03125067949295044)
(2, 8, 0.0022681514422098798, 0.03352335294087728)
(2, 9, 0.005322301387786865, 0.038847935199737546)


(power,nf-jnf):  (2, 0)

(power,nf/24-broadhurst(power):  (2, 0)

(power,jnf/24-broadhurst(power):  (2, 0)

(1128) * x^2 * (x^4 - 4*x^3 - 368/141*x^2 + 624/47*x + 1520

In [3]:
polydata=[]
jpolydata=[]
numericalfactors=[]
jnumericalfactors=[]
no=[]
no_broadhurst=[]
no_jbroadhurst=[]
import time
begin=time.time()
for power in [1..100]:
    data=[]
    jdata=[]
    for m in [3..3*power+3]:
        start=time.time()
        jcst=jpowerConstantTerm(m,power)
        cst=reciprocalDeltaDiamondWt12Strike_ConstantTerm(m,power)
        data=data+[(m,cst)]
        jdata=jdata+[(m,jcst)]
        finish=time.time()
        print((power,m,(finish-start)/60,(finish-begin)/60))
    print()
    rl=R.lagrange_polynomial(data)
    jrl=R.lagrange_polynomial(jdata)
    nf=numericalfactor(rl)
    jnf=numericalfactor(jrl)
    rl=rl.factor()
    jrl=jrl.factor()
    numericalfactors=numericalfactors+[nf/24]
    jnumericalfactors=jnumericalfactors+[jnf/24]
    if nf!=jnf:no=no+[power]
    if nf/24!=broadhurst(power):no_broadhurst=no_broadhurst+[power]
    if jnf/24!=broadhurst(power):no_jbroadhurst=no_jbroadhurst+[power]
    pair=[power,rl]
    jpair=[power,jrl]
    polydata=polydata+[[pair]]
    jpolydata=jpolydata+[[jpair]]
    print()
    print("(power,nf-jnf): ",(power,nf-jnf));print()
    print("(power,nf/24-broadhurst(power): ",(power,nf/24-broadhurst(power)));print()
    print("(power,jnf/24-broadhurst(power): ",(power,jnf/24-broadhurst(power)));print()
    print("-----------------------------------------------------------------------------------")
print("========================================================================================")
print("jnf!=nf:");print(no);print() # # the list "no" is empty
print("no_broadhurst:");print(no_broadhurst);print() # empty
print("no_jbroadhurst:");print(no_jbroadhurst);print() #empty
print("numericalfactors:");print()
print(numericalfactors);print()
print("jnumericalfactors:");print()
print(jnumericalfactors)
import pickle
wfile = open('/Users/barrybrent/30oct22no3.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
wjfile = open('/Users/barrybrent/30oct22no4.txt','wb')
tj = pickle.dumps(str(jpolydata))
wjfile.write(tj)
wjfile.close()

(1, 3, 0.004234683513641357, 0.004243767261505127)
(1, 4, 0.0020325819651285807, 0.006279265880584717)
(1, 5, 0.004253935813903808, 0.010536285241444905)
(1, 6, 0.0031227986017862957, 0.013662282625834148)


(power,nf-jnf):  (1, 0)

(power,nf/24-broadhurst(power):  (1, 0)

(power,jnf/24-broadhurst(power):  (1, 0)

-----------------------------------------------------------------------------------
(2, 3, 0.0040552655855814615, 0.017741115887959798)
(2, 4, 0.0019530812899271647, 0.01969731648763021)
(2, 5, 0.0043019493420918785, 0.02400253216425578)
(2, 6, 0.003215384483337402, 0.027220900853474936)
(2, 7, 0.0046516338984171545, 0.031876250108083086)
(2, 8, 0.0021572311719258628, 0.034036835034688316)
(2, 9, 0.005512619018554687, 0.03955260117848714)


(power,nf-jnf):  (2, 0)

(power,nf/24-broadhurst(power):  (2, 0)

(power,jnf/24-broadhurst(power):  (2, 0)

-----------------------------------------------------------------------------------
(3, 3, 0.004455053806304931, 0.0440308690071106

(9, 20, 0.0047087470690409345, 0.6547774155934651)
(9, 21, 0.008274149894714356, 0.6630537509918213)
(9, 22, 0.004891598224639892, 0.6679485678672791)
(9, 23, 0.006221914291381836, 0.6741738677024841)
(9, 24, 0.004575387636820475, 0.6787531693776448)
(9, 25, 0.0069823145866394045, 0.6857383330663045)
(9, 26, 0.005491467316945394, 0.691233233610789)
(9, 27, 0.0071588635444641115, 0.6983954151471455)
(9, 28, 0.0052954316139221195, 0.7036941329638163)
(9, 29, 0.006465315818786621, 0.7101628502209981)
(9, 30, 0.006384503841400146, 0.7165505011876424)


(power,nf-jnf):  (9, 0)

(power,nf/24-broadhurst(power):  (9, 0)

(power,jnf/24-broadhurst(power):  (9, 0)

-----------------------------------------------------------------------------------
(10, 3, 0.004184885819753011, 0.720828902721405)
(10, 4, 0.0019870678583780925, 0.7228191018104553)
(10, 5, 0.004594465096791585, 0.7274160663286845)
(10, 6, 0.0032498836517333984, 0.7306692997614542)
(10, 7, 0.005634685357411702, 0.736307434240977)
(10

(13, 37, 0.007482751210530599, 1.4851210673650106)
(13, 38, 0.0057953476905822756, 1.4909191330273945)
(13, 39, 0.008282883961995443, 1.4992053826649985)
(13, 40, 0.00536883274714152, 1.5045775016148886)
(13, 41, 0.007337319850921631, 1.5119181354840596)
(13, 42, 0.00716476837793986, 1.5190861503283182)


(power,nf-jnf):  (13, 0)

(power,nf/24-broadhurst(power):  (13, 0)

(power,jnf/24-broadhurst(power):  (13, 0)

-----------------------------------------------------------------------------------
(14, 3, 0.004666932423909505, 1.5239169001579285)
(14, 4, 0.0021255811055501304, 1.5260458827018737)
(14, 5, 0.004731396834055583, 1.530780549844106)
(14, 6, 0.003290450572967529, 1.5340746998786927)
(14, 7, 0.0051487207412719725, 1.539226734638214)
(14, 8, 0.0024884541829427085, 1.5417186856269836)
(14, 9, 0.005776051680246989, 1.5474980513254801)
(14, 10, 0.00418014923731486, 1.5516815503438315)
(14, 11, 0.005858218669891358, 1.5575506170590718)
(14, 12, 0.0042945146560668945, 1.561848680178

(17, 6, 0.003478817145029704, 2.380956935882568)
(17, 7, 0.005424880981445312, 2.3863918662071226)
(17, 8, 0.0028584480285644533, 2.3892537514368692)
(17, 9, 0.006059761842091878, 2.3953165968259174)
(17, 10, 0.004673083623250325, 2.399993101755778)
(17, 11, 0.006140601634979248, 2.406136349836985)
(17, 12, 0.004499419530232748, 2.4106390357017515)
(17, 13, 0.006406994660695394, 2.4170491973559063)
(17, 14, 0.004811131954193115, 2.421863615512848)
(17, 15, 0.008536934852600098, 2.4304062326749167)
(17, 16, 0.0029284119606018066, 2.433337914943695)
(17, 17, 0.006690367062886556, 2.4400302171707153)
(17, 18, 0.005431564648946127, 2.4454657673835754)
(17, 19, 0.006632649898529052, 2.4521017829577128)
(17, 20, 0.005332585175832113, 2.45743776957194)
(17, 21, 0.007750070095062256, 2.4651899695396424)
(17, 22, 0.005361612637837728, 2.47055534919103)
(17, 23, 0.006684982776641845, 2.477243733406067)
(17, 24, 0.004965813954671224, 2.4822139342625937)
(17, 25, 0.007463534673055013, 2.4896809339

(19, 54, 0.007487082481384277, 3.427547732988993)
(19, 55, 0.009717718760172526, 3.437269167105357)
(19, 56, 0.006151680151621501, 3.4434246818224588)
(19, 57, 0.010171182950337728, 3.4535991152127585)
(19, 58, 0.007455182075500488, 3.461057750384013)
(19, 59, 0.009675002098083496, 3.4707362691561383)
(19, 60, 0.008720850944519043, 3.4794612844785053)


(power,nf-jnf):  (19, 0)

(power,nf/24-broadhurst(power):  (19, 0)

(power,jnf/24-broadhurst(power):  (19, 0)

-----------------------------------------------------------------------------------
(20, 3, 0.004254464308420817, 3.4840237816174824)
(20, 4, 0.0022339463233947755, 3.4862608313560486)
(20, 5, 0.005099050203959147, 3.4913631518681845)
(20, 6, 0.003572221597035726, 3.494939653078715)
(20, 7, 0.0061816175778706866, 3.5011297345161436)
(20, 8, 0.0029161492983500164, 3.5040494481722515)
(20, 9, 0.0063485860824584964, 3.510401900609334)
(20, 10, 0.0048049012819925945, 3.5152097503344217)
(20, 11, 0.006712516148885091, 3.521925830841

...

(98, 216, 1.222076912720998, 5242.67868834734)
(98, 217, 1.6243742465972901, 5244.303066281477)
(98, 218, 1.2287331819534302, 5245.531803266207)
(98, 219, 1.5316045999526977, 5247.0634124159815)
(98, 220, 1.2906651178995767, 5248.3540811181065)
(98, 221, 1.4931176821390788, 5249.847202551365)
(98, 222, 1.2539904197057088, 5251.101196650664)
(98, 223, 1.4962947328885396, 5252.59749503533)
(98, 224, 1.0535463174184163, 5253.651045016448)
(98, 225, 1.6591917157173157, 5255.310240582625)
(98, 226, 1.2570992509524028, 5256.567343481382)
(98, 227, 1.4838075637817383, 5258.051154796282)
(98, 228, 1.2714919686317443, 5259.32265021801)
(98, 229, 1.5134722669919332, 5260.836126283804)
(98, 230, 1.1821468154589334, 5262.018277148406)
(98, 231, 1.6578116377194723, 5263.676092553139)
(98, 232, 1.1331594030062357, 5264.80925595363)
(98, 233, 1.5994370500246684, 5266.408696734905)
(98, 234, 1.3440630197525025, 5267.752763434251)
(98, 235, 1.6878304799397787, 5269.440597514312)
(98, 236, 1.25844314893

(99, 87, 1.457960299650828, 5439.8034311334295)
(99, 88, 0.9580061992009481, 5440.761441047986)
(99, 89, 1.4296774824460348, 5442.191122233868)
(99, 90, 1.1244138479232788, 5443.315539749464)
(99, 91, 1.463541865348816, 5444.779085350036)
(99, 92, 1.0584061861038208, 5445.837495235602)
(99, 93, 1.483449367682139, 5447.320948000749)
(99, 94, 0.9746697982152303, 5448.29562151432)
(99, 95, 1.4049468676249186, 5449.700572184722)
(99, 96, 0.8006152510643005, 5450.501189831893)
(99, 97, 1.3620667616526285, 5451.863261198998)
(99, 98, 1.103294352690379, 5452.9665597359335)
(99, 99, 1.4461482048034668, 5454.412711552779)
(99, 100, 1.199573314189911, 5455.612288530668)
(99, 101, 1.3223276336987813, 5456.9346200346945)
(99, 102, 1.0542614181836447, 5457.988886284828)
(99, 103, 1.3161145488421122, 5459.30500446558)
(99, 104, 0.9639214356740315, 5460.268929600716)
(99, 105, 1.5727698008219402, 5461.841703236103)
(99, 106, 0.9862210830052693, 5462.827928066254)
(99, 107, 1.469729216893514, 5464.297

(99, 256, 0.7108659664789836, 5670.011806031068)
(99, 257, 1.6359251499176026, 5671.6477349162105)
(99, 258, 1.385358448823293, 5673.0330975333845)
(99, 259, 1.6853898485501608, 5674.718491399288)
(99, 260, 1.5078322172164917, 5676.226327184836)
(99, 261, 1.7762777169545492, 5678.0026086171465)
(99, 262, 1.3791020353635153, 5679.381714185079)
(99, 263, 1.7055826663970948, 5681.0873006820675)
(99, 264, 1.3413939833641053, 5682.428698416551)
(99, 265, 1.7253134449323018, 5684.1540154298145)
(99, 266, 1.413209617137909, 5685.567228782177)
(99, 267, 1.804101550579071, 5687.371334048113)
(99, 268, 1.3552618980407716, 5688.726599633694)
(99, 269, 1.6770192861557007, 5690.403622535864)
(99, 270, 1.3727451801300048, 5691.776371363799)
(99, 271, 1.619338051478068, 5693.395713202159)
(99, 272, 1.154400368531545, 5694.550117349625)
(99, 273, 1.7386249621709189, 5696.288747215271)
(99, 274, 1.3414481321970622, 5697.630198999246)
(99, 275, 1.7486883838971456, 5699.378890967369)
(99, 276, 1.51630331

(100, 122, 1.1867170651753745, 5869.265242481231)
(100, 123, 1.50453439950943, 5870.769780782858)
(100, 124, 1.2005293885866801, 5871.970313902696)
(100, 125, 1.5659350355466206, 5873.536252717177)
(100, 126, 1.225870168209076, 5874.762127284209)
(100, 127, 1.4433442195256552, 5876.205474499861)
(100, 128, 0.646947451432546, 5876.8524254481)
(100, 129, 1.5879559159278869, 5878.440386664867)
(100, 130, 1.2063920338948568, 5879.646782433987)
(100, 131, 1.4212434013684592, 5881.0680289347965)
(100, 132, 1.3552301049232482, 5882.423262870312)
(100, 133, 1.520454216003418, 5883.943721032143)
(100, 134, 1.2636156360308328, 5885.207340451082)
(100, 135, 1.6178821325302124, 5886.825226950646)
(100, 136, 1.2249600807825725, 5888.050191648801)
(100, 137, 1.5236964344978332, 5889.573891850313)
(100, 138, 1.2952950636545817, 5890.8691908160845)
(100, 139, 1.4653619170188903, 5892.33455641667)
(100, 140, 1.3090487162272135, 5893.64360880057)
(100, 141, 1.5741961359977723, 5895.217808568477)
(100, 1

(100, 287, 1.8827317873636882, 6115.475292817751)
(100, 288, 1.1751570185025533, 6116.650453734398)
(100, 289, 1.893916380405426, 6118.544373865922)
(100, 290, 1.4355627655982972, 6119.979940267404)
(100, 291, 1.8235888004302978, 6121.8035349488255)
(100, 292, 1.3685626188913982, 6123.172101084391)
(100, 293, 1.7256517171859742, 6124.89775706927)
(100, 294, 1.4822166522343954, 6126.379977369308)
(100, 295, 1.8154539545377095, 6128.195435118675)
(100, 296, 1.3284686009089153, 6129.523908150196)
(100, 297, 1.8291189153989156, 6131.353030765056)
(100, 298, 1.3669387499491374, 6132.719973083337)
(100, 299, 1.840924616654714, 6134.560901498795)
(100, 300, 1.4931428154309592, 6136.054050799211)
(100, 301, 1.8526132822036743, 6137.906667602062)
(100, 302, 1.2998073816299438, 6139.206478750706)
(100, 303, 1.8747632503509521, 6141.081245732307)


(power,nf-jnf):  (100, 0)

(power,nf/24-broadhurst(power):  (100, 0)

(power,jnf/24-broadhurst(power):  (100, 0)

------------------------------------